In [1]:
import json
import requests
import datetime
import pandas as pd
import urllib.request
import codecs
import numpy as np
from six.moves import urllib

In [2]:
#eligibility criteria
volume_cutoff = 0.1
percent_float_cutoff = 0.1

#month of rebalance
old_month = 'june'
new_month = 'july'

In [3]:
#change call to new filepath
asset_db = pd.read_excel('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/utils/rebalance_tools/csvs\\Bletchley Rebalance '+new_month+' 2019.xlsx',\
                         sheetname='assets',index_col='Ticker')
#change call to new filepath
#coinmarketcap_db = pd.read_excel('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/utils/rebalance_tools/csvs\\Bletchley Rebalance '+new_month+' 2019.xlsx',\
#                         sheetname='coinmarketcap',index_col='Ticker')

In [4]:
#grab all assets from Messari
#deprecated as they have too many!
#url = "https://data.messari.io/api/v1/assets?with-profiles"
#response = (urllib.request.urlopen(url).read())
#response = response.decode('utf-8')
#d = json.loads(response)
#df = pd.DataFrame(d['data'])
#df = df.set_index((df['symbol']))

In [5]:
df = asset_db
#ticker mapping issues between Bletchley DB and OnChainFX
df.loc['VEN'] = df.loc['VET']
df.index = map(lambda x: x.lower(), df.index)

In [6]:
url_messari = "https://data.messari.io/api/v1/assets/btc/metrics"
response_messari = (urllib.request.urlopen(url_messari).read())
response_messari = response_messari.decode('utf-8')
d_messari = json.loads(response_messari)
df_all = pd.Series()

In [7]:
#get Messari data given a ticker
#don't actually use the category or sector data
def get_messari_data(ticker):
    url_messari = "https://data.messari.io/api/v1/assets/{}/metrics".format(ticker)
    response_messari = (urllib.request.urlopen(url_messari).read())
    response_messari = response_messari.decode('utf-8')
    d_messari = json.loads(response_messari)
    df_all = pd.Series()
    df_all['Y2050 Supply'] = (d_messari['data']['supply']['y_2050'])
    df_all['Available Supply'] = (d_messari['data']['supply']['circulating'])
    df_all['Supply % Issued'] = df_all['Available Supply'] / (df_all['Y2050 Supply'])
    df_all['Price'] = (d_messari['data']['market_data']['price_usd'])
    #df_all['Est Volume'] = ((d_messari['data']['market_data']['volume_last_24_hours']) * 30)
    df_all['Marketcap'] = df_all['Price'] * df_all['Y2050 Supply']
    df_all['Current Marketcap'] = df_all['Price'] * df_all['Available Supply']
    df_all['Category'] = (d_messari['data']['misc_data']['categories'][0])
    df_all['Sectors'] = (d_messari['data']['misc_data']['sectors'][0])
    try:
        df_all['Est Volume'] = ((d_messari['data']['market_data']['volume_last_24_hours']) * 30)
    except:
        pass
    try:
        df_all['Category2'] = (d_messari['data']['misc_data']['categories'][1])
    except:
        pass
    try:
        df_all['Sectors2'] = (d_messari['data']['misc_data']['sectors'][1])
    except:
        pass
    return df_all

In [8]:
#master function to get all Messari data and put in dictionary
metrics_log = dict.fromkeys(df.index)
for t in df.index:
    try:
        metrics = get_messari_data(t)
        metrics_log[t] = metrics
        df.loc[t,('Y2050 Supply')] = metrics['Y2050 Supply']
        df.loc[t,('Available Supply')] = metrics['Available Supply']
        df.loc[t,('Supply % Issued')] = metrics['Supply % Issued']
        df.loc[t,('Price')] = metrics['Price']
        df.loc[t,('Marketcap')] = metrics['Marketcap']
        df.loc[t,('Current Marketcap')] = metrics['Current Marketcap']
        df.loc[t,('Category')] = metrics['Category']
        df.loc[t,('Sectors')] = metrics['Sectors']
        try:
            df.loc[t,('Est Volume')] = metrics['Est Volume']
        except:
            pass
        try:
            df.loc[t,('Category2')] = metrics['Category2']
        except:
            pass
        try:
            df.loc[t,('Sectors2')] = metrics['Sectors2']
        except:
            pass
    except:
        pass

C:\Users\stoin\Anaconda2\envs\py35\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


In [9]:
print(metrics_log['btc'])

Y2050 Supply           2.09863e+07
Available Supply       1.77897e+07
Supply % Issued           0.847679
Price                      11110.5
Marketcap              2.33169e+11
Current Marketcap      1.97652e+11
Category                  Currency
Sectors                   Currency
Est Volume           1158714321720
dtype: object


In [10]:
#t = 'IOTA'
#metrics = get_messari_data(t)
#t = 'IOT'
#print (metrics)
#metrics_log[t] = metrics
#df.loc[t,('Y2050 Supply')] = metrics['Y2050 Supply']
#df.loc[t,('Available Supply')] = metrics['Available Supply']
#df.loc[t,('Est Volume')] = metrics['Est Volume']
#df.loc[t,('Supply % Issued')] = metrics['Supply % Issued']
#df.loc[t,('Price')] = metrics['Price']
#df.loc[t,('Marketcap')] = metrics['Marketcap']
#df.loc[t,('Current Marketcap')] = metrics['Current Marketcap']
#df.loc[t,('Category')] = metrics['Category']
#df.loc[t,('Sectors')] = metrics['Sectors']
#try:
#    df.loc[t,('Est Volume')] = metrics['Est Volume']
#except:
#    pass
#try:
#    df.loc[t,('Category2')] = metrics['Category2']
#except:
#    pass
#try:
#    df.loc[t,('Sectors2')] = metrics['Sectors2']
#except:
#    pass

In [11]:
t = 'VET'
metrics = get_messari_data(t)
metrics_log[t] = metrics
df.loc[t,('Y2050 Supply')] = metrics['Y2050 Supply']
df.loc[t,('Available Supply')] = metrics['Available Supply']
#df.loc[t,('Est Volume')] = metrics['Est Volume']
df.loc[t,('Supply % Issued')] = metrics['Supply % Issued']
df.loc[t,('Price')] = metrics['Price']
df.loc[t,('Marketcap')] = metrics['Marketcap']
df.loc[t,('Current Marketcap')] = metrics['Current Marketcap']
df.loc[t,('Category')] = metrics['Category']
df.loc[t,('Sectors')] = metrics['Sectors']
try:
    df.loc[t,('Est Volume')] = metrics['Est Volume']
except:
    pass
try:
    df.loc[t,('Category2')] = metrics['Category2']
except:
    pass
try:
    df.loc[t,('Sectors2')] = metrics['Sectors2']
except:
   pass

In [12]:
#change Messari data to uppercase to match other sources
df.index = map(lambda x: x.upper(), df.index)

In [13]:
#df.loc['IOT'] = df.loc['IOTA']
#df.loc['VET'] = df.loc['VEN']

In [38]:
df.loc['IOTA']

Coin                               IOTA
Class                                 1
Sector                              NaN
Quality Exchange?                     1
ERC20                                 0
ICO ok?                               1
Eligible                              1
Y2050 Supply                2.77953e+09
Available Supply            2.77953e+09
Supply % Issued                       1
Price                           0.39744
Marketcap                    1.1047e+09
Current Marketcap            1.1047e+09
Category                 Infrastructure
Sectors              Internet of Things
Est Volume                  8.26507e+08
Sectors2                            NaN
Category2                           NaN
Name: IOTA, dtype: object

In [14]:
#read in other data sources
onchainfx_db = df

In [15]:
#remove duplicates
#print (len(coinmarketcap_db))
print (len(onchainfx_db))
print (len(asset_db))
#coinmarketcap_db = coinmarketcap_db[~coinmarketcap_db.index.duplicated(keep='first')]
onchainfx_db = onchainfx_db[~onchainfx_db.index.duplicated(keep='first')]
asset_db = asset_db[~asset_db.index.duplicated(keep='first')]
#print (len(coinmarketcap_db))
print (len(onchainfx_db))
print (len(asset_db))

146
146
145
145


In [40]:
#fix any ticker mapping issues
onchainfx_db = onchainfx_db.rename({'HSR' : 'HC'})
onchainfx_db = onchainfx_db.rename({'HSR' : 'HC'})
onchainfx_db = onchainfx_db.rename({'IOTA' : 'IOT'})
asset_db = asset_db.rename({'IOTA' : 'IOT'})

In [41]:
asset_db

,Coin,Class,Sector,Quality Exchange?,ERC20,ICO ok?,Eligible,Y2050 Supply,Available Supply,Supply % Issued,Price,Marketcap,Current Marketcap,Category,Sectors,Est Volume,Sectors2,Category2
ADA,Cardano,2.0,NaN,1.0,0.0,1.0,1.0,4.201402e+10,2.592707e+10,0.617105,0.083589,3.511915e+09,2.167221e+09,Infrastructure,Smart Contract Platform,6.539403e+09,NaN,NaN
AE,Aeternity,3.0,NaN,1.0,1.0,1.0,1.0,5.036164e+08,2.719622e+08,0.540018,0.468735,2.360625e+08,1.274781e+08,Infrastructure,Smart Contract Platform,1.308312e+09,NaN,NaN
AION,Aion,2.0,NaN,1.0,1.0,1.0,1.0,6.410953e+08,3.184430e+08,0.496717,0.128098,8.212327e+07,4.079204e+07,Infrastructure,Smart Contract Platform,6.729923e+07,Interoperability,NaN
ANT,Aragon,3.0,NaN,1.0,1.0,1.0,1.0,3.960952e+07,3.070660e+07,0.775233,0.550526,2.180609e+07,1.690480e+07,Infrastructure,Onchain Governance,4.653370e+06,NaN,NaN
ARDR,Ardor,2.0,NaN,1.0,0.0,1.0,1.0,9.989995e+08,9.989995e+08,1.000000,0.113189,1.130756e+08,1.130756e+08,Infrastructure,Enterprise & BaaS,5.614946e+07,NaN,NaN
ARK,Ark,2.0,NaN,1.0,0.0,1.0,1.0,3.819367e+08,1.132049e+08,0.296397,0.445749,1.702477e+08,5.046091e+07,Infrastructure,Interoperability,4.319687e+07,NaN,NaN
BAT,BAT,3.0,NaN,1.0,1.0,1.0,1.0,1.500000e+09,1.271156e+09,0.847438,0.290552,4.358278e+08,3.693368e+08,Media and Entertainment,Advertising,9.857852e+08,NaN,NaN
BCD,Bitcoin Diamond,1.0,NaN,1.0,0.0,1.0,1.0,2.100000e+08,1.864929e+08,0.888061,1.142092,2.398392e+08,2.129920e+08,Currency,Currency,2.562235e+08,NaN,NaN
BCH,Bitcoin Cash,1.0,NaN,1.0,0.0,1.0,1.0,2.098243e+07,1.786615e+07,0.851481,405.370812,8.505666e+09,7.242416e+09,Currency,Currency,9.010413e+10,NaN,NaN
BCN,Bytecoin,1.0,NaN,1.0,0.0,0.0,0.0,1.844675e+11,1.840668e+11,0.997828,0.000894,1.648752e+08,1.645172e+08,Currency,Currency,2.164612e+06,Privacy,NaN


In [42]:
#onchainfx_db = pd.read_excel('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/utils/rebalance_tools/csvs\\ocfx_'+new_month+'_2019.xlsx',\
#index_col='Ticker')

In [43]:
#read in previous month Bletchley Data
list_indexes = ['10' , '20' , '40' , 'Total' , 'ETH' , 'Currency' , 'Platform' , 'Application']
indexes_old = dict.fromkeys(list_indexes)
indexes_new_tickers = dict.fromkeys(list_indexes)
indexes_new_file = dict.fromkeys(list_indexes)
for k in indexes_old:
    if (k == 'Currency' or k == 'Platform' or k == 'Application'):
        indexes_old[k] = pd.DataFrame()
        indexes_new_tickers[k] = pd.DataFrame()
        indexes_new_file[k] = pd.DataFrame(columns=['Float'])
        indexes_old[k] = pd.read_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_'+old_month+'_rebal.csv' , index_col=[0] , names=[k,'Adj Float'])
    else:
        indexes_old[k] = pd.DataFrame()
        indexes_new_tickers[k] = pd.DataFrame()
        indexes_new_file[k] = pd.DataFrame(columns=['Float'])
        indexes_old[k] = pd.read_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_'+old_month+'_rebal.csv' , index_col=[0] , names=[k,'Float' , 'Adj Float'])

In [44]:
#main function to create table of all assets
#def create_universe(onchainfx,coinmarketcap,assets):
#    eligibility = pd.DataFrame(columns = ['Marketcap'])
#    eligibility['Marketcap'] = onchainfx['Marketcap']
#    eligibility['Current Marketcap'] = onchainfx['Current Marketcap']
#    eligibility['Float Percent'] = onchainfx['Supply % Issued']
#    eligibility['Float'] = onchainfx['Available Supply']
#    eligibility['Price'] = onchainfx['Price']
#    eligibility['Total Supply'] = onchainfx['Y2050 Supply']
#    eligibility['Float Pass'] = np.where(eligibility['Float Percent']>=0.1, 1, 0)
#    for t in eligibility.index:
#        if t in coinmarketcap.index:
#            eligibility.loc[t,('Volume')] = coinmarketcap.loc[t,('Volume (30d)')]
#        else:
#            print (t,' no volume data!')
#            eligibility.loc[t,('Volume')] = 1
#    eligibility['Volume Ratio'] = eligibility['Volume'] / eligibility['Current Marketcap']
#    eligibility['Volume Pass'] = np.where(eligibility['Volume Ratio']>=volume_cutoff, 1, 0)
#    for t in eligibility.index:
#        if t in assets.index:
#            eligibility.loc[t,('Other Pass')] = assets.loc[t,('Eligible')]
#            eligibility.loc[t,('Class')] = assets.loc[t,('Class')]
#            eligibility.loc[t,('ERC20')] = assets.loc[t,('ERC20')]
#        else:
#            print (t,' no other data!')
#            eligibility.loc[t,('Other Pass')] = 0
#    eligibility['Score'] = eligibility['Volume Pass'] + eligibility['Other Pass'] + eligibility['Float Pass']
#    eligibility['Eligible'] = np.where(eligibility['Score']>=2.99999, 1, 0)
#    check if duplicates
#    count = len(eligibility.index)
#    count1 = len(eligibility.index.unique())
#    if count != count1:
#        print ('duplicate tickers')
#    return eligibility

In [45]:
#main function to create table of all assets
def create_universe(onchainfx,assets):
    eligibility = pd.DataFrame(columns = ['Marketcap'])
    eligibility['Marketcap'] = onchainfx['Marketcap']
    eligibility['Current Marketcap'] = onchainfx['Current Marketcap']
    eligibility['Float Percent'] = onchainfx['Supply % Issued']
    eligibility['Float'] = onchainfx['Available Supply']
    eligibility['Price'] = onchainfx['Price']
    eligibility['Total Supply'] = onchainfx['Y2050 Supply']
    eligibility['Volume'] = onchainfx['Est Volume']
    eligibility['Float Pass'] = np.where(eligibility['Float Percent']>=0.1, 1, 0)
    eligibility['Volume Ratio'] = eligibility['Volume'] / eligibility['Current Marketcap']
    eligibility['Volume Pass'] = np.where(eligibility['Volume Ratio']>=volume_cutoff, 1, 0)
    for t in eligibility.index:
        if t in assets.index:
            eligibility.loc[t,('Other Pass')] = assets.loc[t,('Eligible')]
            eligibility.loc[t,('Class')] = assets.loc[t,('Class')]
            eligibility.loc[t,('ERC20')] = assets.loc[t,('ERC20')]
        else:
            print (t,' no other data!')
            eligibility.loc[t,('Other Pass')] = 0
    eligibility['Score'] = eligibility['Volume Pass'] + eligibility['Other Pass'] + eligibility['Float Pass']
    eligibility['Eligible'] = np.where(eligibility['Score']>=2.99999, 1, 0)
    #check if duplicates
    count = len(eligibility.index)
    count1 = len(eligibility.index.unique())
    if count != count1:
        print ('duplicate tickers')
    return eligibility

In [46]:
#rank assets by 2050 marketcap for selection list
def rank_and_sort(df):
    df['Rank'] = df['Marketcap'].rank(ascending=False).astype(int)
    df_sorted = df.sort_values('Rank')
    return df_sorted

In [47]:
#get list of eligible assets and rank them
#really noisy error logging right now due to Messari having 4000 assets
eligibility_db = create_universe(onchainfx_db,asset_db)
eligible_db = eligibility_db[eligibility_db.Eligible != 0]
ineligible_db = eligibility_db[eligibility_db.Eligible == 0]

duplicate tickers


In [48]:
#make sure I've correctly excluded all of these
double_check = ineligible_db[ineligible_db['Volume Pass'] == 1]
double_check
ineligible_db

,Marketcap,Current Marketcap,Float Percent,Float,Price,Total Supply,Volume,Float Pass,Volume Ratio,Volume Pass,Other Pass,Class,ERC20,Score,Eligible
BCN,1.648752e+08,1.645172e+08,0.997828,1.840668e+11,0.000894,1.844675e+11,2.164612e+06,1,0.013157,0,0.0,1.0,0.0,1.0,0
BLOCK,4.089058e+07,1.105397e+07,0.270330,5.891516e+06,1.876252,2.179375e+07,1.074646e+06,1,0.097218,0,1.0,2.0,0.0,2.0,0
BLZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,1.0,2.0,0.0,1.0,0
BTCP,2.562703e+07,5.741754e+06,0.224051,4.708318e+06,1.219492,2.101452e+07,7.721025e+05,1,0.134472,1,0.0,1.0,0.0,2.0,0
BTM,3.330836e+08,1.591635e+08,0.477848,1.002499e+09,0.158767,2.097944e+09,1.807509e+09,1,11.356304,1,0.0,2.0,0.0,2.0,0
CENNZ,4.888698e+07,3.292818e+07,0.673557,8.082686e+08,0.040739,1.200000e+09,1.205789e+05,1,0.003662,0,0.0,3.0,0.0,1.0,0
CND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,1.0,3.0,0.0,1.0,0
CNX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0.0,3.0,0.0,0.0,0
CTXC,6.088676e+07,3.590870e+07,0.589762,1.497925e+08,0.239723,2.539879e+08,4.062098e+08,1,11.312295,1,0.0,2.0,0.0,2.0,0
DCN,3.610679e+08,1.467685e+07,0.040648,3.251875e+11,0.000045,8.000000e+12,2.886111e+06,0,0.196644,1,1.0,3.0,1.0,2.0,0


In [49]:
#create eligible universe - not sure on why there is a warning
universe = rank_and_sort(eligible_db)

C:\Users\stoin\Anaconda2\envs\py35\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [50]:
universe = universe.rename({'HSR' : 'HC'})
universe = universe.rename({'IOTA' : 'IOT'})
universe

,Marketcap,Current Marketcap,Float Percent,Float,Price,Total Supply,Volume,Float Pass,Volume Ratio,Volume Pass,Other Pass,Class,ERC20,Score,Eligible,Rank
BTC,2.331686e+11,1.976521e+11,0.847679,1.778968e+07,11110.495750,2.098634e+07,1.158714e+12,1,5.862393,1,1.0,1.0,0.0,3.0,1,1
XRP,4.009906e+10,1.706880e+10,0.425666,4.256660e+10,0.400991,1.000000e+11,7.306200e+10,1,4.280440,1,1.0,2.0,0.0,3.0,1,2
ETH,3.996588e+10,3.155763e+10,0.789614,1.067050e+08,295.746439,1.351356e+08,3.865384e+11,1,12.248651,1,1.0,2.0,0.0,3.0,1,3
XLM,1.498015e+10,2.044602e+09,0.136487,1.941988e+10,0.105284,1.422834e+11,1.701455e+10,1,8.321697,1,1.0,1.0,0.0,3.0,1,4
LTC,1.088132e+10,8.102146e+09,0.744592,6.245678e+07,129.724058,8.388054e+07,1.517647e+11,1,18.731423,1,1.0,1.0,0.0,3.0,1,5
BCH,8.505666e+09,7.242416e+09,0.851481,1.786615e+07,405.370812,2.098243e+07,9.010413e+10,1,12.441170,1,1.0,1.0,0.0,3.0,1,6
EOS,7.995815e+09,5.390960e+09,0.674223,9.212659e+08,5.851688,1.366412e+09,1.239191e+11,1,22.986469,1,1.0,2.0,0.0,3.0,1,7
BSV,4.142322e+09,3.524955e+09,0.850961,1.785499e+07,197.421315,2.098214e+07,2.365348e+10,1,6.710293,1,1.0,1.0,0.0,3.0,1,8
LINK,3.861909e+09,1.351668e+09,0.350000,3.500000e+08,3.861909,1.000000e+09,2.760245e+09,1,2.042103,1,1.0,3.0,1.0,3.0,1,9
BNB,3.807610e+09,4.668360e+09,1.226060,1.411755e+08,33.067777,1.151456e+08,9.746414e+09,1,2.087760,1,1.0,3.0,1.0,3.0,1,10


In [51]:
#create ten index membership
ten_auto = universe[universe['Rank'] <= 8]
indexes_new_tickers['10'] = pd.DataFrame()
indexes_new_tickers['10'] = ten_auto
ten_band = universe[universe['Rank'] <= 12]
ten_band = ten_band[ten_band['Rank'] > 8]
count_ten_members = len(ten_auto)
for t in ten_band.index:
    if t in indexes_old['10'].index:
        if count_ten_members < 10:
            indexes_new_tickers['10'] = indexes_new_tickers['10'].append(ten_band.loc[t])
            count_ten_members += 1
            print ('Retained' , t)
        else:
            print ('10 full! No room for' , t)
for t in ten_band.index:
    if t not in indexes_old['10'].index:
        if count_ten_members < 10:
            print ('New add' , t)
            indexes_new_tickers['10'] = indexes_new_tickers['10'].append(ten_band.loc[t])
            count_ten_members += 1
        else:
            print ('10 full! Do not add ' , t)

Retained ADA
Retained TRX
10 full! Do not add  LINK
10 full! Do not add  BNB


In [52]:
#create twenty index membership
twenty_auto = universe[universe['Rank'] <= 26]
for t in twenty_auto.index:
    if t in indexes_new_tickers['10'].index:
        twenty_auto = twenty_auto.drop(t)
#remove all rows in indexes_new_tickers['10']
indexes_new_tickers['20'] = pd.DataFrame()
indexes_new_tickers['20'] = twenty_auto
twenty_band = universe[universe['Rank'] <= 34]
twenty_band = twenty_band[twenty_band['Rank'] > 26]
count_twenty_members = len(twenty_auto)
for t in twenty_band.index:
    if t in indexes_old['20'].index:
        if count_twenty_members < 20:
            indexes_new_tickers['20'] = indexes_new_tickers['20'].append(twenty_band.loc[t])
            count_twenty_members += 1
            print ('Retained' , t)
        else:
            print ('20 full! No room for' , t)
for t in twenty_band.index:
    if t not in indexes_old['20'].index:
        if count_twenty_members < 20:
            print ('New add' , t)
            indexes_new_tickers['20'] = indexes_new_tickers['20'].append(twenty_band.loc[t])
            count_twenty_members += 1
        else:
            print ('20 full! Do not add ' , t)

Retained BTG
Retained BAT
Retained MANA
Retained LSK
20 full! Do not add  QTUM
20 full! Do not add  HOT
20 full! Do not add  OMG
20 full! Do not add  ZIL


In [53]:
#create forty index membership
forty_auto = universe[universe['Rank'] <= 60]
for t in forty_auto.index:
    if t in indexes_new_tickers['10'].index:
        forty_auto = forty_auto.drop(t)
    if t in indexes_new_tickers['20'].index:
        forty_auto = forty_auto.drop(t)
indexes_new_tickers['40'] = pd.DataFrame()
indexes_new_tickers['40'] = forty_auto
forty_band = universe[universe['Rank'] <= 80]
forty_band = forty_band[forty_band['Rank'] > 60]
count_forty_members = len(forty_auto)
for t in forty_band.index:
    if t in indexes_old['40'].index:
        if count_forty_members < 40:
            indexes_new_tickers['40'] = indexes_new_tickers['40'].append(forty_band.loc[t])
            count_forty_members += 1
            print ('Retained' , t)
        else:
            print ('40 full! No room for' , t)
for t in forty_band.index:
    if t not in indexes_old['40'].index:
        if count_forty_members < 40:
            indexes_new_tickers['40'] = indexes_new_tickers['40'].append(forty_band.loc[t])
            count_forty_members += 1
            print ('New add' , t)
        else:
            print ('40 full! Do not add ' , t)

Retained WTC
Retained SNT
Retained MCO
Retained XVG
Retained THETA
Retained PIVX
Retained STORJ
Retained POWR
Retained STRAT
Retained GNT
40 full! No room for AION
40 full! No room for POLY
40 full! Do not add  ARDR
40 full! Do not add  EDO
40 full! Do not add  IOTX
40 full! Do not add  ENG
40 full! Do not add  LRC
40 full! Do not add  WAN
40 full! Do not add  CVC
40 full! Do not add  NMR


In [54]:
#construct total index from 10 + 20 + 40
indexes_new_tickers['Total'] = indexes_new_tickers['10'].append(indexes_new_tickers['20']).append(indexes_new_tickers['40'])

In [55]:
#construct various sector indexes from total index
indexes_new_tickers['ETH'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['ERC20'] == 1]
indexes_new_tickers['Currency'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['Class'] == 1]
indexes_new_tickers['Platform'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['Class'] == 2]
indexes_new_tickers['Application'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['Class'] == 3]

In [56]:
#print out membership changes for checking and blog
for k in indexes_old:
    print (k , 'adds' , indexes_new_tickers[k].index.difference(indexes_old[k].index).values)
    print (k , 'deletes' , indexes_old[k].index.difference(indexes_new_tickers[k].index).values)
    print (k , 'count members:' , len(indexes_new_tickers[k]))
    if indexes_new_tickers[k].index.duplicated(keep='first').any() == True:
        print ('Duplicates if True' , indexes_new_tickers[k].index.duplicated(keep='first'))
    print ("")
sectors = len(indexes_new_tickers['Currency'])+len(indexes_new_tickers['Platform'])+len(indexes_new_tickers['Application'])
print ('Sector count members:' , sectors)

Total adds ['ARK' 'GXS' 'HOT']
Total deletes ['AION' 'ICX' 'POLY']
Total count members: 70

40 adds ['ARK' 'GXS' 'HOT']
40 deletes ['AION' 'ICX' 'POLY']
40 count members: 40

10 adds []
10 deletes []
10 count members: 10

Platform adds ['ARK' 'HOT']
Platform deletes ['AION']
Platform count members: 28

20 adds []
20 deletes []
20 count members: 20

Currency adds []
Currency deletes []
Currency count members: 19

ETH adds []
ETH deletes ['AION' 'POLY']
ETH count members: 20

Application adds ['GXS']
Application deletes ['ICX' 'POLY']
Application count members: 23

Sector count members: 70


In [57]:
#write off new index memberships for use in rebalance
for k in indexes_new_tickers:
    indexes_new_tickers[k].to_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_pre_'+new_month+'.csv',columns=['Float'])